# Chicken and egg in molecular metadynamics
Create protein folding landmarks from scratch



In [2]:
from pypdb import get_pdb_file

id="1L2Y"
pdb=get_pdb_file(id)

f=open(id + ".pdb","w")
f.write(pdb)
f.close()

In [3]:
import nglview as nv

v = nv.NGLWidget()
v.add_component(id + ".pdb")
v.clear()
v.add_representation('cartoon', selection='all', color='green')
v

NGLWidget()

In [7]:
# number of steps to twist a dihedral
nsteps = 12

# number of iterations; alltogether niter * nsteps random conformers are generated
#niter = 200
niter = 10


import math
import pybel
from pyDOE import lhs
from pypdb import describe_pdb
import matplotlib.pyplot as plt
import os

pdb = id + ".pdb"
nres = int(describe_pdb(id)['nr_residues'])

if not os.path.exists('my_folder'):
    os.mkdir('my_folder')
    
out='work/conf%d.pdb'

conf = 1
rgs = []

mols = list(pybel.readfile('pdb',pdb))
mol=mols[0]

# if I knew SMARTS better, oxygens would not have been counted
pattern = '[NH3]-[CH]-C(=O)'
for _ in range(nres-1):
    pattern += '-N-C-C(=O)'
    
sp = pybel.Smarts(pattern)

match = sp.findall(mol)

if len(match) != 1:
    print("backbone pattern not matched")
    raise
    
match = match[0]
    
def set_phi(mol,match,r,phi):
    a1 = mol.atoms[4*(r-1)-2].OBAtom
    a2 = mol.atoms[4*(r-1)].OBAtom
    a3 = mol.atoms[4*(r-1)+1].OBAtom
    a4 = mol.atoms[4*(r-1)+2].OBAtom
    mol.OBMol.SetTorsion(a1,a2,a3,a4,phi/180*math.pi)
    
def set_psi(mol,match,r,phi):
    a1 = mol.atoms[4*(r-1)].OBAtom
    a2 = mol.atoms[4*(r-1)+1].OBAtom
    a3 = mol.atoms[4*(r-1)+2].OBAtom
    a4 = mol.atoms[4*(r-1)+4].OBAtom
    mol.OBMol.SetTorsion(a1,a2,a3,a4,phi/180*math.pi)
    
def rg(mol):
    n = len(mol.atoms)
    
    x = [mol.atoms[i].coords for i in range(n)]
    
    mass = [mol.atoms[i].exactmass for i in range(n)]
    xm = [(m*i,m*j,m*k) for (i,j,k),m in zip(x,mass)]
    tmass = sum(mass)
    rr = sum(mi*i+mj*j+mk*k for (i,j,k),(mi,mj,mk) in zip(x,xm))
    mm = sum((sum(i)/tmass)**2 for i in zip(*xm))
    return math.sqrt(rr/tmass - mm)

rgs = []

for _ in range(niter):
    phi = lhs(nres - 2, nsteps)
    psi = lhs(nres - 2, nsteps)

    for s in range(nsteps):
        for r in range(2,nres):
            set_phi(mol,match,r,phi[s][r-2] * 360)
            set_psi(mol,match,r,psi[s][r-2] * 360)
            
        fn = out % conf
        mol.write('pdb',fn,overwrite=True)
# slooooooow, don't run by default
#        rgs.append(rg(mol))
        conf += 1

if rgs:
    print(rgs)

    fig, ax = plt.subplots()
    n, bins, patches = ax.hist(rgs, 30)
    ax.set_title('Radius of gyration')
    plt.show()



In [5]:
import nglview as nv

v = nv.show_url("work/conf12.pdb")
v.clear()
v.add_representation('cartoon',selection='all',color='green')
v

NGLWidget()

In [16]:
# number of cores to use (all if empty)

ncores=

# the most likely minimization parameters to change; rest is in the template file
minim_mdp = '''
emtol       = 100.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform
'''

!cp ../minim.mdp.template minim.mdp
f=open('minim.mdp','a')
f.write(minim_mdp)
f.close()

!bash ../minim.sh {ncores}


In [18]:
baf=14
!echo {baf}

14


In [ ]:
import math

